# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

%load_ext dotenv
%dotenv 

In [4]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [7]:
import os
from glob import glob

# Write your code below.

PRICE_DATA = os.getenv("PRICE_DATA")

parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("ticker")


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [53]:
# Write your code below.

dd_shift = dd_px.groupby("ticker", group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1     = x["Close"].shift(1),
        Adj_Close_lag_1 = x["Adj Close"].shift(1)))

/var/folders/jt/hkf67kpx6wb_ht6232cq6dw00000gn/T/ipykernel_10028/1126741003.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby("ticker", group_keys=False).apply(


In [47]:
dd_rets = dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1,
    Adj_Returns = lambda x: x['Adj Close']/x['Adj_Close_lag_1'] - 1)


In [51]:
dd_rets = dd_shift.assign(
    Hi_lo_range = lambda x: x['High'] - x['Low'])

In [56]:
dd_feat = dd_shift.assign(
    returns     = lambda d: d["Close"]     / d["Close_lag_1"]     - 1,
    adj_returns = lambda d: d["Adj Close"] / d["Adj_Close_lag_1"] - 1,
    hi_lo_range = lambda d: d["High"]      - d["Low"])

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [63]:
pandas_df = dd_feat.compute()
pandas_df

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,adj_returns,hi_lo_range
ticker,,,,,,,,,,,,,,
A,2015-01-02,41.180000,41.310001,40.369999,40.560001,38.553932,1529200.0,A.csv,2015,NaN,NaN,NaN,NaN,0.940002
A,2015-01-05,40.320000,40.459999,39.700001,39.799999,37.831524,2041800.0,A.csv,2015,40.560001,38.553932,-0.018738,-0.018738,0.759998
A,2015-01-06,39.810001,40.020000,39.020000,39.180000,37.242188,2080600.0,A.csv,2015,39.799999,37.831524,-0.015578,-0.015578,1.000000
A,2015-01-07,39.520000,39.810001,39.290001,39.700001,37.736473,3359700.0,A.csv,2015,39.180000,37.242188,0.013272,0.013272,0.520000
A,2015-01-08,40.240002,40.980000,40.180000,40.889999,38.867619,2116300.0,A.csv,2015,39.700001,37.736473,0.029975,0.029975,0.799999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,2004-06-28,19.270000,19.520000,18.280001,19.000000,17.401590,256400.0,ZEUS.csv,2004,19.219999,17.603077,-0.011446,-0.011446,1.240000
ZEUS,2004-06-29,19.200001,19.900000,19.100000,19.900000,18.225878,123300.0,ZEUS.csv,2004,19.000000,17.401590,0.047368,0.047369,0.799999
ZEUS,2004-06-30,19.940001,20.420000,19.900000,20.420000,18.702133,186000.0,ZEUS.csv,2004,19.900000,18.225878,0.026131,0.026131,0.520000


In [72]:
pandas_df = pandas_df.sort_values(["ticker"])

In [73]:
pandas_df.groupby("ticker")["returns"].rolling(10).mean()

ticker  ticker
A       A              NaN
        A              NaN
        A              NaN
        A              NaN
        A              NaN
                    ...   
ZEUS    ZEUS      0.001594
        ZEUS     -0.006110
        ZEUS      0.003339
        ZEUS      0.010365
        ZEUS      0.005329
Name: returns, Length: 333757, dtype: float64

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Probably not; since we're looking at the last 10 days and not the whole data set, we're not using up a lot of space + requiring a lot of data, so Pandas is convenient. Dask would be better if the data took up too much memory.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.